In [3]:
from tqdm.notebook import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import pandas as pd


def get_oil_info(soup, gu_name):

    os_info ={}

    ## 주유소 이름
    os_name = soup.find_all(class_="header")
    os_info["이름"] = os_name[0].find(id = "os_nm").text

    ## 주유소 구
    os_info["구"] = gu_name

    ## 주유소 주소
    os_addr = soup.find_all(id = "rd_addr")
    os_info["주소"] = os_addr[0].text

    ## 주유소 상표
    os_property = soup.find_all(id = "poll_div_nm")
    os_info["상표"] = os_property[0].text

    ## 휘발유 가격
    gasoline_price = soup.find_all( id = "infoTbody")
    os_info["휘발유 가격"] = gasoline_price[0].find(id = "b027_p").text

    ## 경유 가격
    disel_price = soup.find_all(id = "infoTbody")
    os_info["경유 가격"] = disel_price[0].find(id = "d047_p").text

    ## 부가 기능
    add_info = soup.find_all(class_ = "service")
    for info in add_info[0].find_all("img"):
        if not "off" in info["src"]:
            os_info[info["alt"]] = "o"
        else:
            os_info[info["alt"]] = "x"    

    return os_info


def make_df(os_data):
    
    os_df = pd.DataFrame(os_data)
    print(os_df)
    os_df.to_csv("../../data/os_df.csv")
    


if __name__ == "__main__":

    url = "https://www.opinet.co.kr/searRgSelect.do"
    driver = webdriver.Chrome(service=Service("../../../driver/chromedriver-linux64/chromedriver"))
    
    driver.get(url)
    driver.get(url)

    sido_name = driver.find_element(By.ID, "SIDO_NM0")
    si_list = sido_name.find_elements(By.TAG_NAME , "option")
    sido_list = []
    for idx, si_name in enumerate(si_list):
        sido_list.append(si_name.get_attribute("value"))

    sido_list = sido_list[1:]
    sido_name.send_keys(sido_list[0])


    sigungu = driver.find_element(By.ID, "SIGUNGU_NM0")
    sigungu_list = sigungu.find_elements(By.TAG_NAME, "option")

    sigungu_nm_list = []
    for idx, sigungu_name in enumerate(sigungu_list):
        sigungu_nm_list.append(sigungu_name.get_attribute("value"))

    os_data = []
    for sigungu_name in tqdm(sigungu_nm_list[1:]):
        sigungu = driver.find_element(By.ID, "SIGUNGU_NM0")
        sigungu.send_keys(sigungu_name)

        gasoline = driver.find_element(By.XPATH, '''//*[@id="os_layer2"]/p/a''')
        gasoline.click()
        os_num = driver.find_element(By.XPATH, '''//*[@id="totCnt"]''').text
        
        for idx in range(1,int(os_num) + 1):
            oilstation = driver.find_element(By.XPATH, '''//*[@id="body1"]/tr[{0}]/td[1]/a'''.format(idx))
            oilstation.click()
            time.sleep(1)

            page = driver.page_source
            soup = BeautifulSoup(page, "html.parser")

            os_data.append(get_oil_info(soup, sigungu_name))
        
        time.sleep(3)

    make_df(os_data)
    driver.close()    




  0%|          | 0/25 [00:00<?, ?it/s]

                      이름    구                         주소        상표 휘발유 가격  \
0            (주)보성 세곡주유소  강남구      서울 강남구  헌릉로 731 (세곡동)     SK에너지  1,729   
1    HD현대오일뱅크㈜직영 산성셀프주유소  강남구             서울 강남구 헌릉로 730  HD현대오일뱅크  1,739   
2                  방죽주유소  강남구      서울 강남구 밤고개로 215 (율현동)     GS칼텍스  1,757   
3                자곡셀프주유소  강남구      서울 강남구 밤고개로 120 (자곡동)     SK에너지  1,761   
4    HD현대오일뱅크㈜직영 도곡셀프주유소  강남구  서울 강남구  남부순환로 2718 (도곡2동)  HD현대오일뱅크  1,769   
..                   ...  ...                        ...       ...    ...   
434             (주)기지에너지  중랑구      서울 중랑구 용마산로 716 (신내동)     S-OIL  1,759   
435                신내주유소  중랑구      서울 중랑구 용마산로 705 (신내동)     SK에너지  1,773   
436              신일셀프주유소  중랑구        서울 중랑구 상봉로 58 (망우동)     SK에너지  1,789   
437               용마로주유소  중랑구      서울 중랑구 용마산로 309 (면목동)     SK에너지  1,798   
438                범아주유소  중랑구        서울 중랑구 동일로 881 (묵동)     S-OIL  1,849   

     경유 가격 세차장 충전소 경정비 편의점 24시영업  
0    1,625   o   x   x   x     x  
1    

OSError: Cannot save file into a non-existent directory: '../../data'